In [1]:
from data import *

Using TensorFlow backend.


# Data Augmentasi

Pembuatan data augmentasi menggunakan library **keras.preprocessing.image.ImageDataGenerator** yang bisa menggabungkan Deep Neural Network image dan label secara bersamaan.

## Tentukan parameter data generator
Gunakan parameter dengan metode random dictionary seperti dibawah ini.

In [2]:
data_gen_args = dict(rotation_range = 0.05,
                    width_shift_range = 0.01,
                    height_shift_range = 0.01,
                    shear_range = 0.05,
                    zoom_range = 0.05,
                    horizontal_flip = True,
                    fill_mode = 'nearest')
myGenerator = trainGenerator(600, 'data/train', 'image', 'label', data_gen_args, save_to_dir = "data/train/aug")

## Membuat data augmentasi ke folder **'data/train/aug/'**

In [3]:
num_batch = 2
for i, batch in enumerate(myGenerator):
    print(i)
    if(i>=(num_batch-1)):
        break

Found 600 images belonging to 1 classes.
Found 600 images belonging to 1 classes.
0
1


## Membuat .npy data (data augmentasi yang sudah dikompres)
Butuh memori tambahan untuk menyimpan data ini

In [4]:
image_arr,mask_arr = geneTrainNpy("data/train/aug/","data/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

## Masking Test Data

In [54]:
def masking(img, mask, height=432, width=532, color = yellow):
    '''.'''
    #mask_out = np.zeros((512, 470, 3), dtype = 'uint8') #phantom
    mask_out = np.zeros((height, width, 3), dtype = 'uint8') #GE
    img = np.zeros((height, width, 3), dtype = 'uint8')#cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) # kalau shapenya (432,532) atau grayscale
    for i in range(mask.shape[0]-1):
        for j in range(mask.shape[1]-1):
            if (mask[i,j] >= 30):
                mask_out[i,j,:] = mask[i,j] 
                mask_out[i,j,:] = color
                img[i,j,:] = mask_out[i,j,:]
    segmented = img
    return segmented

In [57]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

results_path = 'data/test/'
results_results = 'results/'
list_file = os.listdir(results_path)
masked = np.zeros((len(list_file)//2, 432, 532, 3), dtype ='uint8')

for i in range(len(list_file)//2):
    masked[i] = masking(np.zeros((432, 532, 3), dtype = 'uint8'), io.imread(results_path+str(i)+'_predict.png'), color = white)

    if (i % 10 == 0):
        clear_output(wait=True)
    else:
        print('Creating... ' + results_path+str(i)+'_predict.png')

    io.imsave(os.path.join(results_results,"%d.png"%i), masked[i])

Creating... data/test/231_predict.png
Creating... data/test/232_predict.png
Creating... data/test/233_predict.png
Creating... data/test/234_predict.png
Creating... data/test/235_predict.png
Creating... data/test/236_predict.png
Creating... data/test/237_predict.png


# CARA CONVERT PNG TO MP4

In [ ]:
ffmpeg -r 30 -f image2 -i %d.png -vcodec libx264 -crf 15  -pix_fmt yuv420p test.mp4

# CARA MENGGABUNGKAN 2 VIDEO YANG SAMA

In [ ]:
ffmpeg -i Yoga60.avi -i aqua.mp4 -filter_complex "[0:v]pad=iw*2:ih[int]; [int][1:v]overlay=W/2:0[vid]" -map "[vid]" -c:v libx264 -crf 15 output2.mp4

## DICE SIMILARITY COEFFICIENT

In [28]:
def dice(im1, im2, empty_score=1.0):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

In [58]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

path_predict = './results/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
dsc = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)-1):
    img1 = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    dsc[i] = dice(img1, img2)
    temp = temp + dsc[i]
    #print(dsc[i])
hasil = temp/len(list_predict)
print(hasil)

0.9349252204703714


## JACCARD SIMILARITY COEFFICIENT

In [59]:
import numpy as np


def jaccard(im1, im2):
    """
    Computes the Jaccard metric, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    jaccard : float
        Jaccard metric returned is a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
    
    Notes
    -----
    The order of inputs for `jaccard` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")
    
    intersection = np.logical_and(im1, im2)

    union = np.logical_or(im1, im2)

    return intersection.sum() / float(union.sum())

In [60]:
import os
from os import listdir
from skimage import io
from IPython.display import clear_output

path_predict = './results/'
path_acuan = './data/Ground Truth/'

list_predict = os.listdir(path_predict)
list_acuan = os.listdir(path_acuan)
dsc = np.zeros(len(list_predict), dtype = float)
temp = 0

for i in range(len(list_predict)-1):
    img1 = cv2.cvtColor(cv2.imread(path_predict+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    img2 = cv2.cvtColor(cv2.imread(path_acuan+str(i)+'.png'), cv2.COLOR_RGB2GRAY)
    dsc[i] = jaccard(img1, img2)
    temp = temp + dsc[i]
    #print(dsc[i])
hasil = temp/len(list_predict)
print(hasil)

0.8836660731418027
